# DataSave Parse
I decided to just save the results of all my GPU runs as pickle files, with one file for each run. This notebook is for parsing those files and plotting the results.

In [ ]:
import os # For interacting with directories and files
import re # For regular expressions
import pickle
import numpy as np

def grab_data(top_directory: str = "DataSaves"):
    # List of filenames (for example)
    filenames = os.listdir(top_directory)
    pattern = r'.*N=(\d+)_p=(\d+).*\.pkl'

    # Finding all matches
    matches = [re.search(pattern, filename) for filename in filenames]
    matches = [match for match in matches if match]

    # Extracting the filenames and the values of N and p
    result = [(match.group(0), int(match.group(1)), int(match.group(2))) for match in matches]
    # Sort results by N and p
    sorted_result = sorted(result, key=lambda x: (x[1], x[2]))

    max_N = max(item[1] for item in sorted_result)
    max_p = max(item[2] for item in sorted_result)

    N_range = range(0,max_N+1)
    p_range = range(0,max_p+1)
    # N changes with column, p changes with row
    N_mesh, p_mesh = np.meshgrid(N_range, p_range)

    return_dict = {
        "N_range" : N_range,
        "p_range" : p_range,
        "N_mesh" : N_mesh,
        "p_mesh" : p_mesh,
    }


    z_value_names = ["expectation", "overlap", "runtime", "num_QAOA_calls"]
    # For each kind of data needed, loop over the files and
    # gather the values in a matrix
    for value_name in z_value_names:
        z_matrix = np.full((max_N+1, max_p+1), np.nan)
        # Extracting data from the sorted pickle files
        for filename, N, p in sorted_result:
            file_path = os.path.join("DataSaves", filename)
            with open(file_path, 'rb') as f:
                data = pickle.load(f)

                # CHECK THAT THE INDEX ORDER IS CORRECT
                z_matrix[p, N] = data[value_name]
        return_dict[value_name] = z_matrix

    return return_dict




# Plotting the Data
Naive implementation that doesn't do anything specific for each plot.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# For interactive plot
%matplotlib widget 
#%matplotlib inline

# Grab data from saved files
return_dict = grab_data()
N_mesh = return_dict["N_mesh"]
p_mesh = return_dict["p_mesh"]

z_value_names = ["expectation", "overlap", "runtime", "num_QAOA_calls"]
for value_name in z_value_names:
    z_matrix = return_dict[value_name]

    # Create a figure
    fig = plt.figure()
    # Add 3D subplot
    ax = fig.add_subplot(111, projection='3d')
    # Plot the surface
    surf = ax.plot_surface(N_mesh, p_mesh, z_matrix, cmap='viridis')
    # Add a color bar which maps values to colors
    fig.colorbar(surf)
    # Add labels
    ax.set_xlabel('Problem Size (N)')
    ax.set_ylabel('QAOA Depth (p)')
    ax.set_zlabel(value_name)
    # Show plot
    plt.show()



# Better Plotting
This time, I will make special graphs for each value type.



# Expectation
Given the results I have seen, a ribbon plot might be good for expectation.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

# Grab data from saved files
return_dict = grab_data()
N_range = return_dict["N_range"]
p_mesh = return_dict["p_mesh"]
expectations = return_dict["expectation"]
expectation_mins = np.min(expectations, axis=0)
expectation_maxs = np.max(expectations, axis=0)

fig, ax = plt.subplots(dpi=1200)

# Plot the ribbon
ax.fill_between(N_range, expectation_mins, expectation_maxs, color='skyblue', alpha=0.5, label='Expectation Range')
ax.plot(N_range, expectation_mins, color='blue', linestyle='--', label='Max')
ax.plot(N_range, expectation_maxs, color='blue', linestyle='--', label='Min')
plt.xlabel('Problem Size (N)')
plt.ylabel('Expectation')
ax.set_xticks(range(0, 21, 5))
plt.title('Expectation vs Problem Size (and # QAOA Layers)')
plt.legend()
plt.show()

# Expectation 2
Maybe the ribbon plot isn't so good, and it's better to just plot many lines

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline
%matplotlib widget

# Grab data from saved files
return_dict = grab_data()
N_range = return_dict["N_mesh"]
p_range = return_dict["p_mesh"]

matrix = return_dict["expectation"]
#print(matrix.shape)

cmap = plt.get_cmap('viridis')

#cmap = plt.get_cmap('viridis')
#colors = [cmap(i) for i in np.linspace(0, 1, len(N_range))]
# Create a figure
fig, ax = plt.subplots()
ax.set_xlabel("Expectation")
ax.set_ylabel('QAOA Depth (p)')
for (i, N) in enumerate(range(5)):
#for (i, N) in enumerate(N_range):
    c_index = -1 + (i / len(N_range))
    c_index = 0 if (i < 5) else 1_000_000
    color = "red" if (i < 5) else "blue"
    #print(color)
    #print(c_index)
    print(matrix[:,N])
    ax.plot(p_range, matrix[:,N])

plt.show()


## Overlap Plotting
For overlap, it is better to plot on log scale

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

# Grab data from saved files
return_dict = grab_data()
N_range = return_dict["N_range"]
p_mesh = return_dict["p_mesh"]
matrix = return_dict["overlap"]

# Create a figure
fig = plt.figure()
# Add 3D subplot
ax = fig.add_subplot(111, projection='3d')
# Plot the surface
surf = ax.plot_surface(N_mesh, p_mesh, np.log10(matrix), cmap='viridis')
# Add a color bar which maps values to colors
fig.colorbar(surf)
# Add labels
ax.set_xlabel('Problem Size (N)')
ax.set_ylabel('QAOA Depth (p)')
ax.set_zlabel("Log10(Overlap)")
# Show plot
plt.show()



# Number of QAOA Calls

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

# Grab data from saved files
return_dict = grab_data()
N_mesh = return_dict["N_mesh"]
p_mesh = return_dict["p_mesh"]


z_matrix = return_dict["num_QAOA_calls"]

# Create a figure
fig = plt.figure(dpi=1200)
# Add 3D subplot
ax = fig.add_subplot(111, projection='3d')
# Plot the surface
surf = ax.plot_surface(N_mesh, p_mesh, z_matrix, cmap='viridis')
# Add a color bar which maps values to colors
fig.colorbar(surf)
# Add labels
ax.set_xlabel('Problem Size (N)', fontsize=14, labelpad=5)
ax.set_ylabel('QAOA Depth (p)', fontsize=14, labelpad=5)
ax.set_zlabel("# QAOA Calls", fontsize=14, labelpad=10)
ax.view_init(elev=30, azim=230)  # Elevation angle: 30 degrees, Azimuthal angle: 60 degrees
ax.set_xticks(range(0, 21, 5))
ax.set_yticks(range(0, 21, 5))
# Show plot
ax.set_box_aspect(aspect=None, zoom=0.8)
plt.tight_layout()
plt.show()
